# Convolutional Neural Network 

In [287]:
import warnings
import json
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import tensorflow as tf
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
#%matplotlib inline

In [288]:
categories = ['business','entertainment','general','health',
            'technology', 'sports']
cat_num_map ={
    'business': 0,
    'entertainment':1,
    'general': 2,
    'health' : 3,
    'technology': 4,
    'sports' : 5
}
news_store = list()
with open('./news_baseline.json','r',encoding="utf8") as f :
    ls = f.readlines()
    for l in ls:
        news = json.loads(l)
        #combine sci with tech 
        if(news['category']=="science"):
            news['category']="technology"
        # combine title and description
        # to-dos
        news_store.append(news)
    
news_store = pd.DataFrame(news_store)
news_store.drop('_id', axis=1, inplace=True)

In [289]:
# prepare training and testing data
data = news_store.sample(frac=1.0)
test_num = int(0.2 * data.shape[0]) # 20% percent of test data
data_test = data[:test_num]
data_train = data[test_num:]
X_test_raw = data_test['description']
Y_test_str = data_test['category']
X_train_raw = data_train['description']
Y_train_str = data_train['category']
Y_test = list()
Y_train = list()
# convert str to numbers
for d in Y_test_str:
    Y_test.append(cat_num_map[d])

for d in Y_train_str:
    Y_train.append(cat_num_map[d])
    
print('Test data num: {}\nTraining data num: {}'.format(test_num,data.shape[0]-test_num))

Test data num: 1251
Training data num: 5008


In [290]:
#Tokenize words: get rid of stop words and punctuation + word stemming.
stop = stopwords.words('english') + list(string.punctuation)
tok_corpus = list()
for sent in news_store['description']:
    tokenized_sent = [d for d in nltk.RegexpTokenizer(r'\w+').tokenize(sent.lower()) if d not in stop]
    for i, w in enumerate(tokenized_sent):
        tokenized_sent[i] = SnowballStemmer('english').stem(w)
    tok_corpus.append(tokenized_sent) 


In [332]:
X_train_tok = list()
X_test_tok  = list()

for sent in X_train_raw:
    tokenized_sent = [d for d in nltk.RegexpTokenizer(r'\w+').tokenize(sent.lower()) if d not in stop]
    for i, w in enumerate(tokenized_sent):
        tokenized_sent[i] = SnowballStemmer('english').stem(w)
    X_train_tok.append(tokenized_sent) 

for sent in X_test_raw:
    tokenized_sent = [d for d in nltk.RegexpTokenizer(r'\w+').tokenize(sent.lower()) if d not in stop]
    for i, w in enumerate(tokenized_sent):
        tokenized_sent[i] = SnowballStemmer('english').stem(w)
    X_test_tok.append(tokenized_sent) 

In [334]:
w2v_model = gensim.models.Word2Vec(tok_corpus, size=100, window=5, min_count=1, workers=4)

In [372]:
# convert input to vector
X_train = list()
X_test  = list()

for doc in X_train_tok:
    doc_vec = list()
    for i in range(30):
        try:
            w = doc[i] # i-th word(w)
            w_vec = w2v_model.wv[w]
            doc_vec.append(w_vec)
        except Exception:
            doc_vec.append(np.zeros(100))
    X_train.append(doc_vec)
    

for doc in X_test_tok:
    doc_vec = list()
    for i in range(30):
        try:
            w = doc[i] # i-th word(w)
            w_vec = w2v_model.wv[w]
            doc_vec.append(w_vec)
        except Exception:
            doc_vec.append(np.zeros(100))
    X_test.append(doc_vec)
X_train = np.asarray(X_train)
X_test  = np.asarray(X_test)   

In [374]:
'''Start CNN'''
def cnn_model_fn(features, labels, mode):

    input_layer = tf.reshape(features["x"], [-1,30,100,1])
    
    conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=10,
            kernel_size=[5,5],
            padding='SAME',
            activation=tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(
            inputs=conv1,
            pool_size=[2,2],
            strides=2,
            padding='SAME')
    
    conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=10,
            kernel_size=[5,5],
            padding='SAME',
            activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(
            inputs=conv2,
            pool_size=[2,2],
            strides=2)
    
    # fully-connected layers with dropout
    pool2_flat = tf.reshape(pool2,[-1,7*25*10])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate = 0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=6)
    
    
    
    predictions={
        "classes": tf.argmax(input=logits,axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor") 
    }
    #PREDICT
    if (mode == tf.estimator.ModeKeys.PREDICT):
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=6)
    print(onehot_labels.shape)
    loss = tf.losses.softmax_cross_entropy(
                                onehot_labels=onehot_labels, logits=logits)
    #loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logtis)
    #TRAIN
    if mode == tf.estimator.ModeKeys.TRAIN:
        print('yo')
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    #Else mode == EVAL 
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    

In [270]:
# for test only
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


In [272]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=100)


clf = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./mnist_convnet_model_adm")


train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

clf.train(
    input_fn=train_input_fn,
    steps=2000,
    hooks=[logging_hook])

    
# if __name__ == "__main__":
#     tf.app.run()


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028384363D30>, '_log_step_count_steps': 100, '_session_config': None, '_task_type': 'worker', '_save_summary_steps': 100, '_service': None, '_tf_random_seed': None, '_task_id': 0, '_is_chief': True, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './mnist_convnet_model_adm', '_num_ps_replicas': 0}
(100, 10)
yo
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./mnist_convnet_model_adm\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.10485634  0.1123668   0.08455718  0.0917285   0.10072833  0.09407595
   0.11891305  0.10236228  0.09924661  0.0911651 ]
 [ 0.10643921  0.09895497  0.08778913  0.09859087  0.07620083  0.09623069
   0.12009132  0.12695132  0.0885743

INFO:tensorflow:loss = 2.33048, step = 1
INFO:tensorflow:global_step/sec: 5.80279
INFO:tensorflow:probabilities = [[ 0.00012344  0.00742697  0.813465    0.00216996  0.09977411  0.00067197
   0.00103257  0.03641367  0.03678473  0.00213759]
 [ 0.00297612  0.00000039  0.00000368  0.0000656   0.0000014   0.98413348
   0.00002866  0.00000288  0.01270985  0.00007787]
 [ 0.0000798   0.9677518   0.00013264  0.00266231  0.00682843  0.00053276
   0.00008784  0.01971658  0.00045286  0.00175506]
 [ 0.00441037  0.00008941  0.00087999  0.00000059  0.00071025  0.00001258
   0.99387395  0.00001536  0.00000736  0.00000008]
 [ 0.00000576  0.0004757   0.99283534  0.00023972  0.00000079  0.0000127
   0.00000019  0.00620203  0.00022254  0.0000052 ]
 [ 0.00000146  0.00000259  0.00375272  0.99620831  0.          0.00000952
   0.00000014  0.0000003   0.0000249   0.00000001]
 [ 0.00010328  0.00000001  0.00000005  0.02619352  0.00000008  0.96974987
   0.00000005  0.00005496  0.00381498  0.00008327]
 [ 0.0003634

INFO:tensorflow:loss = 0.299264, step = 101 (17.270 sec)
INFO:tensorflow:global_step/sec: 4.10588
INFO:tensorflow:probabilities = [[ 0.00118418  0.00006466  0.02434377  0.48057368  0.00000789  0.01656273
   0.00000237  0.00001978  0.47524488  0.00199603]
 [ 0.00002543  0.99887544  0.00005708  0.00000277  0.00015506  0.00000441
   0.00061089  0.00004415  0.00022409  0.00000074]
 [ 0.01467758  0.00002073  0.00012087  0.00000608  0.05533685  0.00016857
   0.92274559  0.00000678  0.00601339  0.00090358]
 [ 0.00000036  0.00000981  0.00177075  0.99604332  0.00002052  0.00032767
   0.00000002  0.00037255  0.00103763  0.00041725]
 [ 0.00001585  0.00000003  0.00004009  0.00002487  0.00000007  0.0000007
   0.          0.99931896  0.0000001   0.00059933]
 [ 0.00279589  0.00000661  0.88707626  0.10507852  0.00153408  0.00005754
   0.00000889  0.00066859  0.0002209   0.00255266]
 [ 0.00011168  0.99480766  0.00001028  0.00010198  0.00321203  0.00002939
   0.00007155  0.00077746  0.00036292  0.000515

INFO:tensorflow:loss = 0.125883, step = 201 (24.354 sec)
INFO:tensorflow:global_step/sec: 4.58118
INFO:tensorflow:probabilities = [[ 0.00026047  0.882788    0.00072583  0.00057498  0.00006171  0.00000029
   0.00000165  0.11117072  0.00417202  0.0002444 ]
 [ 0.00047002  0.00028474  0.00201902  0.36987737  0.00269943  0.43866956
   0.00028398  0.00188511  0.0320745   0.15173618]
 [ 0.00726571  0.00000606  0.97977918  0.00010633  0.00030071  0.00009101
   0.00069051  0.00000095  0.00678054  0.00497896]
 [ 0.00004255  0.00000129  0.00053939  0.00004039  0.00000227  0.00000374
   0.00000449  0.00019989  0.99907303  0.00009294]
 [ 0.00000105  0.00000002  0.00000007  0.00002184  0.00000329  0.99982566
   0.0000328   0.          0.00009245  0.00002289]
 [ 0.0000242   0.00136675  0.00018146  0.00003988  0.99826306  0.00000585
   0.00004047  0.00005739  0.00000199  0.00001901]
 [ 0.00000079  0.0000007   0.00000392  0.00003207  0.00003929  0.00005023
   0.00000456  0.00000296  0.9998073   0.00005

INFO:tensorflow:loss = 0.161965, step = 301 (21.763 sec)
INFO:tensorflow:global_step/sec: 4.88176
INFO:tensorflow:probabilities = [[ 0.00000008  0.00000049  0.00000327  0.0000174   0.00000001  0.00000006
   0.          0.9999001   0.00000023  0.0000783 ]
 [ 0.00001629  0.00000179  0.00003454  0.00001698  0.00000017  0.89373314
   0.00000949  0.00000205  0.10618132  0.00000422]
 [ 0.00001667  0.99536198  0.00005714  0.00000042  0.00273519  0.00000008
   0.00000108  0.00179858  0.00002718  0.0000015 ]
 [ 0.          0.00000006  0.00000036  0.00000002  0.00000036  0.00000001
   0.          0.99999857  0.00000025  0.00000038]
 [ 0.00069827  0.00041711  0.00002081  0.00043272  0.00000841  0.99483782
   0.00233971  0.00000008  0.00119594  0.0000491 ]
 [ 0.00051456  0.010482    0.00126087  0.00022739  0.00314154  0.00300043
   0.96995711  0.00000555  0.01140342  0.00000716]
 [ 0.00000133  0.00000019  0.00030343  0.00396984  0.00000001  0.00089993
   0.00000023  0.00000092  0.99481624  0.00000

...

INFO:tensorflow:loss = 0.0370832, step = 1501 (21.383 sec)
INFO:tensorflow:global_step/sec: 4.81329
INFO:tensorflow:probabilities = [[ 0.99999893  0.          0.00000102  0.          0.          0.          0.
   0.00000002  0.          0.00000005]
 [ 0.00000001  0.99999988  0.          0.          0.00000013  0.          0.
   0.00000005  0.00000001  0.        ]
 [ 0.          0.00000017  0.00000008  0.0000037   0.          0.          0.
   0.99999607  0.          0.00000001]
 [ 0.00000001  0.          0.          0.00000926  0.          0.99970144
   0.00004153  0.          0.00024711  0.00000056]
 [ 0.00005594  0.00000496  0.00000029  0.00002459  0.00000083  0.0001749
   0.00000678  0.000001    0.99970919  0.00002144]
 [ 0.00000053  0.00000898  0.00000267  0.00004501  0.00000005  0.00000037
   0.          0.99991822  0.00000051  0.00002375]
 [ 0.00000002  0.00022958  0.00000176  0.00011232  0.01358417  0.00000014
   0.00000001  0.98492211  0.00001413  0.00113587]
 [ 0.          0.0

INFO:tensorflow:loss = 0.041906, step = 1601 (20.777 sec)
INFO:tensorflow:global_step/sec: 4.74534
INFO:tensorflow:probabilities = [[ 0.          0.00000001  0.          0.          0.99999988  0.          0.
   0.00000001  0.00000008  0.        ]
 [ 0.          0.          0.00000003  0.99999809  0.          0.00000007
   0.          0.00000103  0.00000027  0.00000043]
 [ 0.          0.00000038  0.99998558  0.00000041  0.          0.          0.
   0.00000005  0.00001373  0.        ]
 [ 0.00000023  0.99997556  0.00000005  0.00000001  0.00000212  0.          0.
   0.00001929  0.00000254  0.00000025]
 [ 0.000004    0.99874014  0.00001096  0.00000174  0.00083114  0.00000583
   0.00002418  0.00002778  0.00034475  0.00000942]
 [ 0.          0.          0.00000496  0.00000001  0.99885285  0.          0.
   0.00000526  0.00002585  0.00111101]
 [ 0.          0.00000001  0.00000006  0.00001131  0.          0.          0.
   0.99998629  0.00000001  0.0000023 ]
 [ 0.00000004  0.00000004  0.00000

INFO:tensorflow:loss = 0.00237273, step = 1701 (21.160 sec)
INFO:tensorflow:global_step/sec: 4.30826
INFO:tensorflow:probabilities = [[ 0.00000109  0.00000024  0.99999273  0.00000456  0.          0.          0.
   0.00000067  0.00000071  0.        ]
 [ 0.00225922  0.00004118  0.77364117  0.12044801  0.00000011  0.00006745
   0.          0.10344854  0.00000442  0.00008999]
 [ 0.          0.          0.00000033  0.00000001  0.00000001  0.00000059
   0.          0.00000005  0.99999893  0.0000001 ]
 [ 0.00000013  0.00000677  0.00000444  0.00003873  0.00000012  0.00000006
   0.          0.99986947  0.00000041  0.00007984]
 [ 0.00000072  0.00000037  0.          0.00000354  0.0000808   0.00000008
   0.          0.00000032  0.00004744  0.99986672]
 [ 0.          0.          0.          1.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.00000025  0.          0.          0.
   0.99999976  0.          0.00000001]
 [ 0.00000002  0

INFO:tensorflow:loss = 0.00952767, step = 1801 (23.349 sec)
INFO:tensorflow:global_step/sec: 4.73566
INFO:tensorflow:probabilities = [[ 0.00000001  0.00000009  0.00000004  0.00000128  0.00002025  0.00000018
   0.          0.00001344  0.00014506  0.9998197 ]
 [ 0.          0.          0.          0.00000061  0.00000098  0.00000006
   0.          0.00000754  0.00000178  0.99998903]
 [ 0.          0.0000007   0.          0.          0.9999963   0.          0.
   0.0000026   0.00000001  0.0000004 ]
 [ 0.          0.          0.0000004   0.00000865  0.00000577  0.0000049
   0.          0.00005763  0.00011774  0.99980491]
 [ 0.00000216  0.99999356  0.00000002  0.          0.0000011   0.          0.
   0.00000296  0.00000009  0.00000017]
 [ 0.00000091  0.00004835  0.00000246  0.00032482  0.00022817  0.00000247
   0.          0.00198252  0.00004232  0.99736804]
 [ 0.00000001  0.          0.00000001  0.00000184  0.00001274  0.00000041
   0.          0.00004921  0.00133752  0.99859828]
 [ 0.0000

INFO:tensorflow:loss = 0.00763234, step = 1901 (20.893 sec)
INFO:tensorflow:Saving checkpoints for 2000 into ./mnist_convnet_model_adm\model.ckpt.
INFO:tensorflow:Loss for final step: 0.0296081.


In [273]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": eval_data},
  y=eval_labels,
  num_epochs=1,
  shuffle=False)
eval_results = clf.evaluate(input_fn=eval_input_fn)
print(eval_results)

(?, 10)
INFO:tensorflow:Starting evaluation at 2018-05-05-20:30:45
INFO:tensorflow:Restoring parameters from ./mnist_convnet_model_adm\model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2018-05-05-20:30:54
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9876, global_step = 2000, loss = 0.0334609
{'accuracy': 0.98760003, 'loss': 0.033460908, 'global_step': 2000}
